# Dispatch with Energy Storage

In this case, we will show the usage of energy storage included dispatch.

In AMS, ``ESD1`` is a dispatch model for energy storage, which has a corresponding
dynamic model ``ESD1`` in ANDES.

In [1]:
import pandas as pd

import ams

In [2]:
ams.config_logger(stream_level=20)

A small-size PJM 5-bus case with ESD1 is used in this example.

In [3]:
sp = ams.load(ams.get_case('5bus/pjm5bus_demo.xlsx'),
              setup=True,
              no_output=True)

Parsing input file "/Users/jinningwang/work/ams/ams/cases/5bus/pjm5bus_demo.xlsx"...
Input file parsed in 0.1092 seconds.
Zero Line parameters detected, adjusted to default values: rate_b, rate_c.
Parameters c2, c1 are altered to 0 as they are associated with ESD1 for following GCost: GCost_5
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0022 seconds.


The model parameters can be inspected as follow.

In [4]:
sp.ESD1.as_df()

,idx,u,name,bus,gen,Sn,gammap,gammaq,SOCmin,SOCmax,...,SOCend,En,EtaC,EtaD,cesdc,cesdd,tdc,tdd,tdc0,tdd0
uid,,,,,,,,,,,,,,,,,,,,,
0,ESD1_1,1.0,ESD1_1,1,PV_2,100.0,1.0,1.0,0.1,0.9,...,0.5,100.0,1.0,1.0,0.2,0.1,0.0,0.0,0.0,0.0


`RTEDES` extends RTED to include energy storage.

Note that mixed integer linear programming (MILP) requires
capable solvers such as Gurobi or CPLEX.
They might require extra installation and have their own license.

The example here only aims to show the usage of RTEDES.
More details can be found at [CVXPY - Choosing a solver](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver).

In [5]:
sp.RTEDES.run(solver='SCIP')

Building system matrices
Parsing OModel for <RTEDES>
Evaluating OModel for <RTEDES>
Finalizing OModel for <RTEDES>
<RTEDES> initialized in 0.0234 seconds.
<RTEDES> solved as optimal in 0.0320 seconds, converged in -1 iteration with SCIP.


True

Note that, in RTED, the time interval is 5/60 [H] by default, and the
dispatch model has been adjusted accordingly.

In [6]:
sp.RTEDES.SOC.v

array([0.49999999])

Similarly, multi-period dispatch ``EDES`` and ``UCES`` are also available.
They have 1 [H] time interval by default.

In [7]:
sp.EDES.config.t

1

In [8]:
sp.EDES.run(solver='SCIP')

Parsing OModel for <EDES>
Evaluating OModel for <EDES>
Finalizing OModel for <EDES>
<EDES> initialized in 0.0207 seconds.
<EDES> solved as optimal in 0.2852 seconds, converged in -1 iteration with SCIP.


True

In [9]:
sp.EDES.SOC.v

array([[0.51443054, 0.5307006 , 0.54861132, 0.5672678 , 0.58632201,
        0.60507792, 0.62164629, 0.63562937, 0.64831982, 0.65986678,
        0.67050532, 0.67936513, 0.68646668, 0.69085368, 0.65212282,
        0.6124508 , 0.57175288, 0.53103739, 0.49081307, 0.45901485,
        0.46518142, 0.47347561, 0.485122  , 0.5       ]])

In [10]:
sp.UCES.run(solver='SCIP')

All generators are online at initial, make initial guess for commitment.
As initial commitment guess, turn off StaticGen: PV_1
Parsing OModel for <UCES>
Evaluating OModel for <UCES>
Finalizing OModel for <UCES>
<UCES> initialized in 0.0221 seconds.
<UCES> solved as optimal in 0.4066 seconds, converged in -1 iteration with SCIP.


True

In [11]:
sp.UCES.SOC.v

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])